In [2]:
import numpy as np
import scipy.stats as si
import math

# Parameters
T = 0.25
N = 52
dt = T / N
mu = 0.13
sigma = 0.2
S0 = 49
K = 50
r = 0.05

# Black-Scholes Model
def black_scholes(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = (S * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0))
    return call_price

bs_call_price = black_scholes(S0, K, T, r, sigma)
print(f"Black-Scholes Call Price: {bs_call_price}")

# Monte Carlo Method
def monte_carlo(S, K, T, r, sigma, N_simulations=10000):
    random_returns = np.random.normal((r - 0.5 * sigma ** 2) * T, sigma * np.sqrt(T), N_simulations)
    stock_prices = S * np.exp(random_returns)
    call_payoffs = np.maximum(stock_prices - K, 0)
    call_price = np.exp(-r * T) * np.mean(call_payoffs)
    return call_price

mc_call_price = monte_carlo(S0, K, T, r, sigma)
print(f"Monte Carlo Call Price: {mc_call_price}")

# Binomial Tree Method
def binomial_tree(S, K, T, r, sigma, N):
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)
    price_tree = np.zeros((N+1, N+1))
    
    for i in range(N + 1):
        for j in range(i + 1):
            price_tree[j, i] = S * (u ** (i - j)) * (d ** j)
    
    option_tree = np.zeros_like(price_tree)
    option_tree[:, -1] = np.maximum(0, price_tree[:, -1] - K)
    
    for i in range(N - 1, -1, -1):
        for j in range(0, i + 1):
            option_tree[j, i] = np.exp(-r * dt) * (p * option_tree[j, i + 1] + (1 - p) * option_tree[j + 1, i + 1])
    
    return option_tree[0, 0]

bt_call_price = binomial_tree(S0, K, T, r, sigma, N)
print(f"Binomial Tree Call Price: {bt_call_price}")


Black-Scholes Call Price: 1.7779239451401487
Monte Carlo Call Price: 1.780580050347759
Binomial Tree Call Price: 1.7830110680950624
